If you haven't seen one before, this is a Jupyter notebook. Jupyter is a system that lets us embed code snippets with text for educational purposes. The code in this file is broken into cells which can be executed independently. To run a cell, click on it (the outline should turn green) and hit SHIFT + ENTER. To inspect the value of a variable, just write its name as the last line in the cell.

# Part I: Linearization

The first step of designing the LQR controller for our quadrotor is to linearize the system about the hover state. The equations for the 3D quadrotor, however, are very complicated, and computing the necessary derivatives is too tedious to want to do by hand. Therefore, we will use a computer algebra system (CAS) to save us a lot of time in computing derivatives. Specifically, we will be using the [SymPy](https://docs.sympy.org/latest/index.html) package to help us along the way. If you have experience with Mathematica or MATLAB's Symbolic Toolkit, SymPy offers many of the same features but in a Python interface, and even allows us to convert the symbolic functions that we will derive into efficient numerical ones.

To get started, we will demonstrate how to use SymPy to symbolically linearize the planar quadrotor model, then leave it to you to linearize the 3D model. We also suggest looking through the [brief tutorial](https://docs.sympy.org/latest/tutorial/index.html) given in the SymPy documentation, which will cover most of what you will need for this assignment. Again, the dynamics of the planar quadrotor are:

$$\begin{align}\ddot{x} &= -\frac{u_1}{m}\sin\psi\\ \ddot{z} &= \frac{u_1}{m}\cos\psi - g\\ \ddot{\theta} &= \frac{u_2}{I}\end{align}$$

We begin by importing the functions we need from SymPy.

In [ ]:
import sympy as sp
import numpy as np
from sympy.physics.vector import dynamicsymbols as dynamicsymbols

Next, we define the symbolic variables we need to describe the equations of motion. The function ``dynamicsymbols`` creates symbols that vary in time, i.e. ``dynamicsymbols('x')`` will create a symbol $x(t)$ as opposed to $x$.

In [ ]:
m, g, I, r, t = sp.symbols('m g I r t')
u1, u2  = sp.symbols('u1 u2')
x, y, theta = dynamicsymbols('x y theta')

We also define some variables as shorthand for the time derivatives of our state variables.

In [ ]:
x_dot = sp.diff(x, t)
y_dot = sp.diff(y, t)
theta_dot = sp.diff(theta, t)

Now, we write out the equations of motion. Note that `sp.Matrix` is used to create both matrices and vectors in a manner similar to `np.array`.

In [ ]:
state = sp.Matrix([x, y, theta, x_dot, y_dot, theta_dot])
input = sp.Matrix([u1, u2])

dynamics = sp.Matrix([x_dot, y_dot, theta_dot,
                      -u1 / m * sp.sin(theta),
                      u1 / m * sp.cos(theta) - g,
                      u2 / I])

Finally, we differentiate and plug in numerical values via the `subs` method. In this case, the hover state is chosen to be the $0$ vector and the hover input is $u_1 = m g$. We'll leave the parameters of the system as symbolic values for now so we can see the structure of the result.

In [ ]:
A = dynamics.jacobian(state)
B = dynamics.jacobian(input)

In [ ]:
A.subs([(u1, m * g), (theta, 0)])

In [ ]:
B.subs([(theta, 0)])

The last thing we need to discuss is turning the A and B matrices in the previous cells into `np.ndarray` types. The reason we want to do this is that after we replace the remaining system parameters with numerical values, SymPy will represent these values in a way that is very precise, but incredibly inefficient for the numerical work we need to do. Instead, we want to work with NumPy's numerical arrays, i.e. `np.ndarray`. These cannot handle symbolic values like SymPy's matrices and will not operate at same level of precision, but they are much more efficient for computational purposes.

In [ ]:
A_arr = np.array(A.subs([(u1, m * g), (theta, 0), (m, 0.03), (g, 9.81)])).astype(float)

B_arr = np.array(B.subs([(theta, 0), (m, 0.03), (I, 1.419e-5)])).astype(float)

In each of these two lines, we are doing the following. First, we substitute in all the hover state and parameter numerical values as before. Next, we call `np.array` to convert from `sp.Matrix` to `np.ndarray`. However, we still have to tell numpy which data type to use to represent the numerical values in this matrix. To do so, we call the method `np.ndarray.astype()` with the type we want, `float`. You can see these matrices have a different numerical representation now:

In [ ]:
print(A_arr)
print()
print(B_arr)

## Problem 1
Now it's your turn! In the following cell, please write the code to linearize the 3D quadrotor model that was discussed in class. You are free to use the full functionality of SymPy --- the [cross product](https://docs.sympy.org/latest/modules/matrices/matrices.html?highlight=cross#sympy.matrices.matrices.MatrixBase.cross) and [matrix inverse](https://docs.sympy.org/latest/modules/matrices/matrices.html?highlight=cross#sympy.matrices.matrices.MatrixBase.inv) methods will be particularly useful. The function should print $\mathbf{A}$ and $\mathbf{B}$ matrices computed like above as `np.ndarray` variables. Please place your answers in the variables `A_arr` and `B_arr`. As a reminder, the dynamics are given by

$$\begin{align}\ddot{\mathbf{r}} &= \begin{bmatrix}0\\0\\-g\end{bmatrix} + \mathbf{R} \begin{bmatrix}0\\0\\\frac{F_{tot}}{m}\end{bmatrix}\\ \dot{\omega}_{BW} &= \mathbf{I}^{-1} \left(-\omega_{BW} \times \mathbf{I} \omega_{BW} + \begin{bmatrix}M_1\\ M_2\\ M_3\end{bmatrix}\right)\end{align}$$

where $\mathbf{r}$ is the position in the world frame of the center of mass of the quadrotor, i.e.

$$\mathbf{r} = \begin{bmatrix}x\\ y\\ z\end{bmatrix}$$

the rotation matrix $\mathbf{R}$ maps from the body frame to the world frame following the Space 1-2-3 convention, that is

$$\mathbf{R} = \begin{bmatrix}\cos(\theta)\cos(\psi) & \sin(\phi)\sin(\theta)\cos(\psi) - \sin(\psi)\cos(\phi) & \cos(\phi)\sin(\theta)\cos(\psi) + \sin(\psi)\sin(\phi)\\
\cos(\theta)\sin(\psi) & \sin(\phi)\sin(\theta)\cos(\psi) + \cos(\psi)\cos(\phi) & \cos(\phi)\sin(\phi)\sin(\psi) - \cos(\theta)\sin(\phi)\\
-\sin(\theta) & \sin(\phi)\cos(\theta) & \cos(\phi)\cos(\theta)\end{bmatrix}$$

and 

$$\omega_{BW} = \begin{bmatrix}p\\ q\\ r\end{bmatrix}$$

is the angular velocity of the body frame in the world frame, which is related to the rate of change of the Euler angles by

$$\begin{bmatrix}\dot{\phi}\\ \dot{\theta}\\ \dot{\psi}\end{bmatrix} = \begin{bmatrix}1 & \sin(\phi)\tan(\theta) & \cos(\phi)\tan(\theta)\\
         0 & \cos(\phi) & -\sin(\phi)\\
         0 & \frac{\sin(\phi)}{\cos(\theta)} & \frac{\cos(\phi)}{\cos(\theta)}
        \end{bmatrix}\begin{bmatrix}p\\ q\\ r\end{bmatrix}$$

In addition to the hover state and input, you should substitute the following values for the physical parameters of the system:

- $m = 0.03kg$
- $g = 9.81 \frac{m}{s^2}$
- $I_{xx} = 1.4194e-05$
- $I_{yy} = 1.4089e-05$
- $I_{zz} = 2.9741e-05$

Note the full state of the system $\mathbf{x}$ and system inputs $\mathbf{u}$ are the variables, in order:

$$\begin{align}\mathbf{x} = \begin{bmatrix}x\\ y\\ z\\ \phi\\ \theta\\ \psi\\ \dot{x}\\ \dot{y}\\ \dot{z}\\ p\\ q\\ r\end{bmatrix} & & \mathbf{u} = \begin{bmatrix}F_{tot}\\ M_1\\ M_2\\ M_3\end{bmatrix}\end{align}$$

In [ ]:
print(A_arr)
print()
print(B_arr)

# Part II: Calculating Thrust Coefficient

In this part of the assignment, we will calculate the _thrust coefficient_ and _drag coefficient_ of the Crazyflie.

The force, $F_i$, and moment, $M_i$, produced by each motor is given by

$$\begin{align}F_i = k_f \omega_i^2 & & M_i = k_m \omega_i^2\end{align}$$

Here, $\omega_i$ is the rotational speed of the motor, $k_f$ is the thrust coefficient, and $k_m$ is the drag coefficient. The experimental procedure used to find $k_f$ is straightforward. Since $\omega_i^2$ is linearly related to $F_i$ and $\omega_i^2$ can be set to arbitrary values, we can simply measure the thrust produced at different values of $\omega_i$ and perform a _linear regression_, which is another name for calculating the line of best fit.

## Linear Regression Overview

Linear regression is a simple but useful statistical tool. This section will provide a quick overview of the technique in case you have not already seen it in a statistics or linear algebra class.

Consider the following scenario. We have a data set $\{(\mathbf{x}_i, y_i)\}_{i = 1}^N$, where $\mathbf{x}_i \in \mathbb{R}^n$ represents the input to some function and $y_i \in \mathbb{R}$ represents the measured output. We assume that the output is linearly related to the input and that there is some amount of additive random noise $e_i$ representing measurement error, i.e.

$$y_i = \mathbf{a}^{\top}\mathbf{x}_i + e_i$$

Our goal is to characterize the mapping between $\mathbf{x}$ and $\mathbf{y}$ by finding $\mathbf{a}$. To do so, we first recognize that we can write the $N$ constraints given by the previous equation as

$$\mathbf{y} = \mathbf{X}\mathbf{a} + \mathbf{e}$$

where

$$\begin{align}\mathbf{y} = \begin{bmatrix}y_1\\ \vdots \\ y_N\end{bmatrix} & & \mathbf{X} = \begin{bmatrix}\mathbf{x}_1^{\top}\\ \vdots \\ \mathbf{x}_N^{\top}\end{bmatrix} & & \mathbf{e} = \begin{bmatrix}e_1\\ \vdots \\ e_N\end{bmatrix}\end{align}$$.

If $\mathbf{e}$ is known, we could simply attempt to invert $\mathbf{X}$ to compute $\mathbf{a}$. However, since the measurement noise is rarely known, we cannot persue this avenue. Moreover, $\mathbf{X}$ is rarely square and / or of full rank. We often choose $N >> n$ in order to compensate for the fact that some of our measurments will have significant errors. So, instead, we will try to find a value for $\mathbf{a}$ that tries to get "as close as possible" to satisfying

$$\mathbf{y} = \mathbf{X}\mathbf{a}$$.

While there are a number of ways we can define "as close as possible" formally, the one that we are going to pick is the square of the $2$-norm of the difference between the left and right hand sides of the previous equation. This choice is made for a number of practical and analytic reasons that are beyond the scope of this class. This choice corresponds to the optimization problem

$$\underset{\mathbf{a}}{\mathrm{minimize}}\ \frac{1}{2}\|\mathbf{y} - \mathbf{X}\mathbf{a}\|_2^2$$

This kind of optimization problem is known as a _least-squares problem_ and is a kind of _unconstrained quadratic program_. The details of what that means are unimportant, but methods for solving these problems are very well-studied, and methods for doing so are already implemented as part of SciPy.

## Implementation

First, we are going to import the libraries and data that we need. Feel free to use any function in these libraries to help in the assignment.

In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import lsq_linear

npzfile = np.load('quad_data/thrust_data.npz')
thrusts = npzfile['thrusts']
grams = npzfile['grams']

## Problem 2
You will use the `lsq_linear` function to solve the least-squares problem described above. It's documentation is available [here](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.optimize.lsq_linear.html).

First, we need to discuss an implementation detail. Internally, the Crazyflie doesn't work in terms of commanded angular velocities for the rotors but instead in terms of a percentage of the maximum angular velocity of which the rotors are capable. Moreover, this quantity is represented as a 16-bit integer, so values range from $0$ to $2^{16}$, with $0$ corresponding to $0$% angular velocity and $2^{16}$ corresponding to $100$% angular velocity. Therefore, for the purposes of writing a controller, we are really interested in measure the value of $k_f$ for the equation

$$F_{tot} = k_f \sum_{i = 1}^4\tilde{\omega}_i^2$$

where $\tilde{\omega}^2_i$ is the percentage of max. angular velocity represented as a 16-bit integer in the manner just described for the $i$th rotor and $F_{tot}$ is the total thrust in _newtons_.

To collect data, we measured the thrust of the quadrotor on a scape while sweeping $\tilde{\omega}_i^2$ through a series of values three different times. The data from this experimented was just imported in the following format:
- `thrusts`: An `np.ndarray` containing $M$ different percentages of max. thrust to which the rotors were set for data collection _in grams_. You will need to convert to Newtons. _All four rotors were set to this value at the same time_.
- `grams`: An `np.ndarray` of dimension $M$-by-$3$. The value in entry `grams[i, j]` corresponds to the thrust, in grams measured at `thrusts[i]` on the `j`th experimental trial.

Your assignment, is to, in the next cell, compute $k_f$ by finding the best-fit line for this data. It's been found in the literature that, for this model of quadrotor, that

$$\frac{k_f}{k_m} \approx 40.7333$$

Therefore, you should also compute $k_m$ once you find $k_f$. Please put these values in the variables `k_f` and `k_m` respectively.

In [ ]:
k_f = 0
k_m = 0

Finally, run the following cell to check your work. It plots the data along with the best-fit line you found. If these don't line up (pun unintended), you will need to reevaluate your approach.

In [ ]:
%matplotlib notebook
for i in range(grams.shape[1]):
    if i == 0:
        plt.scatter(thrusts, 9.81 * grams[:, i], c='k', s=5, label='Experimental Data')
    else:
        plt.scatter(thrusts, 9.81 * grams[:, i], c='k', s=5)

plt.plot(thrusts, k_f * (2 ** 16) * thrusts, c='r', linewidth=2, label='Fitted Line')
plt.xlabel('PWM [%]')
plt.ylabel('Thrust [N]')
plt.legend()

print(f'k_f = {k_f}\nk_m = {k_m}')

# Part III: Testing out the Crazyflie

In this last part of the assignment, we ask that you assemble and configure the Crazyflie and run a simple test script to make sure everything is functioning before the next lab.

First, assemble your Crazyflie. Instructions are located [here](https://www.bitcraze.io/getting-started-with-the-crazyflie-2-0/). **There is one caveat you should be aware of, however.** Instead of using the short connector pins, you need to use the longer set. This is because we are using the FlowDeck expansion deck, which will provide us with significantly more accurate state measurements. There are instructions for mounting expansion decks on the Crazyflie [here](https://www.bitcraze.io/getting-started-with-expansion-decks/). We suggest you push the pins down until they appear on the other side of the flow deck to ensure a secure connection (the pins can be slid further through the black plastic on the connector but it may take some physical effort).

Once the FlowDeck is installed, run the command `cfclient` in terminal and make sure your drone is on. This software is a GUI for interacting with the Crazyflie. Connect the Crazyflie to your computer by USB and turn it on. Then, hit `Scan` in the upper left of the window and from the drop down menu pick `usb://0` and hit `Connect`. This uses the USB interface to connect to the drone. In the console tab (hit `View -> Tabs -> Console` if it isn't present), you should see the following lines if the FlowDeck is connected and functioning:

```
DECK_CORE: 1 deck(s) found
DECK_CORE: Calling INIT on driver bcFlow2 for deck 0
```

The last thing we have to do is set the radio channel the Crazyflie will operate on and will prevent other groups from sending commands to your drone (or your commands to their drone). Hit `Connect -> Configure 2.X`. Set the radio channel option in the dialog box that appears to **your group number**. Also make sure that the radio bandwith is 2 MBit/s (if it's set to something else, change it to 2 MBit/s). Hit `Write` (if you don't, this won't be saved). Now, change the value of the variable in the cell below to your group number.

In [ ]:
group_number = 80

Now, plug in the CrazyRadio, hit the `Disconnect` button and `Scan` again. You may need to tell VirtualBox to enable the CrazyRadio by right-clicking on the USB icon in the lower right corner of the window. You should see a URI that looks something like `radio://0/<group_number>/2M`. You don't need to connect to it again, and, in fact, you can't command the drone via Python while it's connected to the GUI. However, you should confirm the URL exists.

To check that everything is working properly, run the following cell. It will print a bunch of logging information (specifically the roll, pitch, and yaw of the drone).

In [1]:
# This is an example from the Crazyflie Python API.
# See https://github.com/bitcraze/crazyflie-lib-python/blob/master/examples/basiclogSync.py

import logging
import time

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger

# Only output errors from the logging framework
logging.basicConfig(level=logging.ERROR)


# Initialize the low-level drivers (don't list the debug drivers)
cflib.crtp.init_drivers(enable_debug_driver=False)
# Scan for Crazyflies and use the first one found
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()
print('Crazyflies found:')
for i in available:
    print(i[0])

if len(available) == 0:
    print('No Crazyflies found, cannot run example')
else:
    lg_stab = LogConfig(name='Stabilizer', period_in_ms=10)
    lg_stab.add_variable('stabilizer.roll', 'float')
    lg_stab.add_variable('stabilizer.pitch', 'float')
    lg_stab.add_variable('stabilizer.yaw', 'float')

    cf = Crazyflie(rw_cache='./cache')
    with SyncCrazyflie(available[0][0], cf=cf) as scf:
        with SyncLogger(scf, lg_stab) as logger:
            endTime = time.time() + 10

            for log_entry in logger:
                timestamp = log_entry[0]
                data = log_entry[1]
                logconf_name = log_entry[2]

                print('[%d][%s]: %s' % (timestamp, logconf_name, data))

                if time.time() > endTime:
                    break


Scanning interfaces for Crazyflies...
Crazyflies found:
radio://0/80/2M
Connecting to radio://0/80/2M
Connected to radio://0/80/2M
[5472][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -5.123465538024902, 'stabilizer.pitch': 0.9995155334472656, 'stabilizer.yaw': -74.64348602294922}
[5482][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -4.907101154327393, 'stabilizer.pitch': 1.0123569965362549, 'stabilizer.yaw': -74.73056030273438}
[5492][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -5.1388654708862305, 'stabilizer.pitch': 1.0188322067260742, 'stabilizer.yaw': -74.47305297851562}
[5502][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -4.920216083526611, 'stabilizer.pitch': 1.22148859500885, 'stabilizer.yaw': -74.30548858642578}
[5512][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -5.149296760559082, 'stabilizer.pitch': 0.983049

[6072][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -5.325095176696777, 'stabilizer.pitch': 1.0019444227218628, 'stabilizer.yaw': -70.31478118896484}
[6082][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -5.007354736328125, 'stabilizer.pitch': 1.2002593278884888, 'stabilizer.yaw': -69.98584747314453}
[6092][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -5.207666873931885, 'stabilizer.pitch': 1.094321370124817, 'stabilizer.yaw': -70.04656219482422}
[6102][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -4.9795684814453125, 'stabilizer.pitch': 1.141990303993225, 'stabilizer.yaw': -69.90131378173828}
[6112][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -4.9963860511779785, 'stabilizer.pitch': 1.1772698163986206, 'stabilizer.yaw': -69.80204010009766}
[6122][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.r

[6702][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -4.824120044708252, 'stabilizer.pitch': 1.2080483436584473, 'stabilizer.yaw': -65.48645782470703}
[6712][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -4.864317893981934, 'stabilizer.pitch': 1.1719346046447754, 'stabilizer.yaw': -65.49783325195312}
[6722][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -4.900768756866455, 'stabilizer.pitch': 0.6789236068725586, 'stabilizer.yaw': -66.32783508300781}
[6732][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -5.119340419769287, 'stabilizer.pitch': 0.8987893462181091, 'stabilizer.yaw': -66.0949478149414}
[6742][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -5.154266834259033, 'stabilizer.pitch': 0.9385722875595093, 'stabilizer.yaw': -66.0177993774414}
[6752][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.rol

[7322][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -4.951570987701416, 'stabilizer.pitch': 0.7577548027038574, 'stabilizer.yaw': -62.13909149169922}
[7332][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -5.035357475280762, 'stabilizer.pitch': 1.0865297317504883, 'stabilizer.yaw': -61.81681442260742}
[7342][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -5.222866058349609, 'stabilizer.pitch': 1.094624400138855, 'stabilizer.yaw': -62.00223922729492}
[7352][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -4.905567646026611, 'stabilizer.pitch': 1.2989919185638428, 'stabilizer.yaw': -61.40125274658203}
[7362][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -5.193111896514893, 'stabilizer.pitch': 1.1201844215393066, 'stabilizer.yaw': -61.76756286621094}
[7372][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.ro

[7952][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -5.15069580078125, 'stabilizer.pitch': 1.0105173587799072, 'stabilizer.yaw': -58.52297592163086}
[7962][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -4.913029670715332, 'stabilizer.pitch': 1.2292402982711792, 'stabilizer.yaw': -57.8782958984375}
[7972][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -4.929945945739746, 'stabilizer.pitch': 1.2656978368759155, 'stabilizer.yaw': -57.79707717895508}
[7982][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -5.008718967437744, 'stabilizer.pitch': 1.2253022193908691, 'stabilizer.yaw': -57.86290740966797}
[7992][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -5.00369930267334, 'stabilizer.pitch': 1.1945370435714722, 'stabilizer.yaw': -57.840423583984375}
[8002][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.rol

...

[11062][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -4.73629903793335, 'stabilizer.pitch': 1.1978307962417603, 'stabilizer.yaw': -42.45528793334961}
[11072][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -4.788408279418945, 'stabilizer.pitch': 0.9620869755744934, 'stabilizer.yaw': -42.757835388183594}
[11082][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -5.026707649230957, 'stabilizer.pitch': 0.9771485924720764, 'stabilizer.yaw': -42.95874786376953}
[11092][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -4.8079609870910645, 'stabilizer.pitch': 1.2609364986419678, 'stabilizer.yaw': -42.29494857788086}
[11102][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -4.904421329498291, 'stabilizer.pitch': 1.0809972286224365, 'stabilizer.yaw': -42.60022735595703}
[11112][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabi

[11692][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -5.208474159240723, 'stabilizer.pitch': 1.038686990737915, 'stabilizer.yaw': -40.3030891418457}
[11702][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -4.839273452758789, 'stabilizer.pitch': 1.1741130352020264, 'stabilizer.yaw': -39.90778732299805}
[11712][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -5.134547233581543, 'stabilizer.pitch': 0.949189305305481, 'stabilizer.yaw': -40.282875061035156}
[11722][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -4.891213893890381, 'stabilizer.pitch': 1.116446614265442, 'stabilizer.yaw': -39.89283752441406}
[11732][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -5.168733596801758, 'stabilizer.pitch': 0.9819769859313965, 'stabilizer.yaw': -40.21705627441406}
[11742][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilize

[12322][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -5.0457444190979, 'stabilizer.pitch': 1.1740580797195435, 'stabilizer.yaw': -40.65381622314453}
[12332][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -4.819001197814941, 'stabilizer.pitch': 1.1516166925430298, 'stabilizer.yaw': -40.532344818115234}
[12342][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -4.764729976654053, 'stabilizer.pitch': 1.1024532318115234, 'stabilizer.yaw': -40.43595504760742}
[12352][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -4.747730255126953, 'stabilizer.pitch': 1.1249048709869385, 'stabilizer.yaw': -40.83293914794922}
[12362][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -4.696643829345703, 'stabilizer.pitch': 1.106179118156433, 'stabilizer.yaw': -40.83536911010742}
[12372][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabiliz

[12962][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -13.55267333984375, 'stabilizer.pitch': 7.503203868865967, 'stabilizer.yaw': -51.23252868652344}
[12972][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -15.650582313537598, 'stabilizer.pitch': 5.71242094039917, 'stabilizer.yaw': -35.199981689453125}
[12982][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -16.039766311645508, 'stabilizer.pitch': 5.60066556930542, 'stabilizer.yaw': -35.994510650634766}
[12992][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -16.750822067260742, 'stabilizer.pitch': 3.764176845550537, 'stabilizer.yaw': -24.1865291595459}
[13002][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -17.351499557495117, 'stabilizer.pitch': 4.174806594848633, 'stabilizer.yaw': -27.11149787902832}
[13012][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabiliz

[13602][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -6.782970905303955, 'stabilizer.pitch': 17.14048194885254, 'stabilizer.yaw': -57.61004638671875}
[13612][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -6.89925479888916, 'stabilizer.pitch': 16.82189178466797, 'stabilizer.yaw': -56.73701477050781}
[13622][<cflib.crazyflie.log.LogConfig object at 0x7f399402e518>]: {'stabilizer.roll': -7.256019592285156, 'stabilizer.pitch': 16.49287986755371, 'stabilizer.yaw': -54.7038688659668}
